In [1]:
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings(action='ignore')

In [49]:
df = pd.read_csv('./gurgaon_properties.csv')
df = df.drop(columns = 'Unnamed: 0')
df.head()

,property_type,property_name,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating,property_id
0,house,2 Bedroom House for sale in Laxman Vihar,independent,1.25,78125.0,160.0,Built Up area: 160 (14.86 sq.m.),2,3,0,NaN,"Laxman Vihar, Sector 3a, Laxman Vihar, Gurgaon...",1.0,NaN,undefined,"['Chintapurni Mandir', 'Sheetla Mata Mandir', ...",Best in class property available at laxman vih...,"['1 Light', 'No AC', 'No Bed', 'No Chimney', '...",NaN,"['Environment4 out of 5', 'Lifestyle4 out of 5...",NaN
1,house,3 Bedroom House for sale in Sector 52 Gurgaon,ardee city,2.25,16071.0,1400.0,Plot area 2160(200.67 sq.m.)Built Up area: 180...,3,3,3,Servant Room,"Upper First Floor, Sector 52 Gurgaon, Gurgaon,...",4.0,South-East,0 to 1 Year Old,"['Hanuman Mandir', 'New Life Church', 'Radhakr...",Brand new 3bhk independent upper first floor o...,"['3 Wardrobe', '7 Fan', '1 Exhaust Fan', '3 Ge...","['Private Garden / Terrace', 'Maintenance Staf...","['Environment4 out of 5', 'Safety4 out of 5', ...",NaN
2,flat,3 BHK Flat in Sector 37D Gurgaon,signature global city,1.25,6082.0,2055.0,Carpet area: 2055 (190.92 sq.m.),3,3,3,not available,"Sector 37D Gurgaon, Gurgaon, Haryana",4.0,NaN,Dec 2024,"['Sapphire 83 Mall Sector 83', 'Dwarka Express...",It's a very beautiful property with amazing lo...,"['1 Modular Kitchen', '1 Chimney', '4 AC', 'No...","['Power Back-up', 'Lift(s)', 'Swimming Pool', ...","['Environment4 out of 5', 'Lifestyle4 out of 5...",H69851460
3,house,4 Bedroom House for sale in Palam Vihar,ansal api spanish court,2.90,18626.0,1557.0,Plot area 173(144.65 sq.m.)Built Up area: 1415...,4,4,3,Store Room,"A, Block Ansals Palam Vihar, Palam Vihar, Gurg...",2.0,North,1 to 5 Year Old,"['Palam Vihar Vyapar kendra', 'Palam triangle'...",On sale 173 sq yrds built up house park facing...,"['4 Wardrobe', '10 Fan', '1 Exhaust Fan', '2 G...","['Feng Shui / Vaastu Compliant', 'Maintenance ...","['Environment5 out of 5', 'Lifestyle5 out of 5...",NaN
4,flat,2 BHK Flat in Sector 89 Gurgaon,smart world gems,0.86,7796.0,1103.0,Carpet area: 1103 (102.47 sq.m.),2,3,2,"study room,pooja room,servant room","Sector 89 Gurgaon, Gurgaon, Haryana",4.0,North-East,Within 3 months,"['Sant Soordas Sihi Metro Station', 'Airia Mal...","Park facing, green belt, at very reasonable ra...","['2 Wardrobe', '3 Fan', '1 Exhaust Fan', '3 Ge...","['Security / Fire Alarm', 'Feng Shui / Vaastu ...","['Environment4 out of 5', 'Safety4 out of 5', ...",G66179018


In [50]:
df.shape

(3974, 21)

In [51]:
df.duplicated().sum()

13

In [52]:
df.columns

Index(['property_type', 'property_name', 'society', 'price', 'price_per_sqft',
       'area', 'areaWithType', 'bedRoom', 'bathroom', 'balcony',
       'additionalRoom', 'address', 'floorNum', 'facing', 'agePossession',
       'nearbyLocations', 'description', 'furnishDetails', 'features',
       'rating', 'property_id'],
      dtype='object')

In [53]:
# dropping property_id
df = df.drop(columns='property_id')

In [54]:
df.insert(loc=2, column='sector', value=df['property_name'].str.split('in').str.get(1).str.replace('Gurgaon',''))

In [55]:
df.head()

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
0,house,2 Bedroom House for sale in Laxman Vihar,Laxman Vihar,independent,1.25,78125.0,160.0,Built Up area: 160 (14.86 sq.m.),2,3,0,NaN,"Laxman Vihar, Sector 3a, Laxman Vihar, Gurgaon...",1.0,NaN,undefined,"['Chintapurni Mandir', 'Sheetla Mata Mandir', ...",Best in class property available at laxman vih...,"['1 Light', 'No AC', 'No Bed', 'No Chimney', '...",NaN,"['Environment4 out of 5', 'Lifestyle4 out of 5..."
1,house,3 Bedroom House for sale in Sector 52 Gurgaon,Sector 52,ardee city,2.25,16071.0,1400.0,Plot area 2160(200.67 sq.m.)Built Up area: 180...,3,3,3,Servant Room,"Upper First Floor, Sector 52 Gurgaon, Gurgaon,...",4.0,South-East,0 to 1 Year Old,"['Hanuman Mandir', 'New Life Church', 'Radhakr...",Brand new 3bhk independent upper first floor o...,"['3 Wardrobe', '7 Fan', '1 Exhaust Fan', '3 Ge...","['Private Garden / Terrace', 'Maintenance Staf...","['Environment4 out of 5', 'Safety4 out of 5', ..."
2,flat,3 BHK Flat in Sector 37D Gurgaon,Sector 37D,signature global city,1.25,6082.0,2055.0,Carpet area: 2055 (190.92 sq.m.),3,3,3,not available,"Sector 37D Gurgaon, Gurgaon, Haryana",4.0,NaN,Dec 2024,"['Sapphire 83 Mall Sector 83', 'Dwarka Express...",It's a very beautiful property with amazing lo...,"['1 Modular Kitchen', '1 Chimney', '4 AC', 'No...","['Power Back-up', 'Lift(s)', 'Swimming Pool', ...","['Environment4 out of 5', 'Lifestyle4 out of 5..."
3,house,4 Bedroom House for sale in Palam Vihar,Palam Vihar,ansal api spanish court,2.90,18626.0,1557.0,Plot area 173(144.65 sq.m.)Built Up area: 1415...,4,4,3,Store Room,"A, Block Ansals Palam Vihar, Palam Vihar, Gurg...",2.0,North,1 to 5 Year Old,"['Palam Vihar Vyapar kendra', 'Palam triangle'...",On sale 173 sq yrds built up house park facing...,"['4 Wardrobe', '10 Fan', '1 Exhaust Fan', '2 G...","['Feng Shui / Vaastu Compliant', 'Maintenance ...","['Environment5 out of 5', 'Lifestyle5 out of 5..."
4,flat,2 BHK Flat in Sector 89 Gurgaon,Sector 89,smart world gems,0.86,7796.0,1103.0,Carpet area: 1103 (102.47 sq.m.),2,3,2,"study room,pooja room,servant room","Sector 89 Gurgaon, Gurgaon, Haryana",4.0,North-East,Within 3 months,"['Sant Soordas Sihi Metro Station', 'Airia Mal...","Park facing, green belt, at very reasonable ra...","['2 Wardrobe', '3 Fan', '1 Exhaust Fan', '3 Ge...","['Security / Fire Alarm', 'Feng Shui / Vaastu ...","['Environment4 out of 5', 'Safety4 out of 5', ..."


In [56]:
df['sector'] = df['sector'].str.lower()

In [75]:
df['sector'] = df['sector'].str.replace('sohna', 'sohna road')
df['sector'] = df['sector'].str.replace('dharam colony','sector 12')
df['sector'] = df['sector'].str.replace('krishna colony','sector 7')
df['sector'] = df['sector'].str.replace('suncity','sector 54')
df['sector'] = df['sector'].str.replace('prem nagar','sector 13')
df['sector'] = df['sector'].str.replace('mg road','sector 28')
df['sector'] = df['sector'].str.replace('gandhi nagar','sector 28')
df['sector'] = df['sector'].str.replace('laxmi garden','sector 11')
df['sector'] = df['sector'].str.replace('shakti nagar','sector 11')

df['sector'] = df['sector'].str.replace('baldev nagar','sector 7')
df['sector'] = df['sector'].str.replace('shivpuri','sector 7')
df['sector'] = df['sector'].str.replace('garhi harsaru','sector 17')
df['sector'] = df['sector'].str.replace('imt manesar','manesar')
df['sector'] = df['sector'].str.replace('adarsh nagar','sector 12')
df['sector'] = df['sector'].str.replace('shivaji nagar','sector 11')
df['sector'] = df['sector'].str.replace('bhim nagar','sector 6')
df['sector'] = df['sector'].str.replace('madanpuri','sector 7')

df['sector'] = df['sector'].str.replace('saraswati vihar','sector 28')
df['sector'] = df['sector'].str.replace('arjun nagar','sector 8')
df['sector'] = df['sector'].str.replace('ravi nagar','sector 9')
df['sector'] = df['sector'].str.replace('vishnu garden','sector 105')
df['sector'] = df['sector'].str.replace('bhondsi','sector 11')
df['sector'] = df['sector'].str.replace('surya vihar','sector 21')
df['sector'] = df['sector'].str.replace('devilal colony','sector 9')
df['sector'] = df['sector'].str.replace('valley view estate','gwal pahari')

df['sector'] = df['sector'].str.replace('mehrauli  road','sector 14')
df['sector'] = df['sector'].str.replace('jyoti park','sector 7')
df['sector'] = df['sector'].str.replace('ansal plaza','sector 23')
df['sector'] = df['sector'].str.replace('dayanand colony','sector 6')
df['sector'] = df['sector'].str.replace('sushant lok phase 2','sector 55')
df['sector'] = df['sector'].str.replace('chakkarpur','sector 28')
df['sector'] = df['sector'].str.replace('greenwood city','sector 45')
df['sector'] = df['sector'].str.replace('subhash nagar','sector 12')

df['sector'] = df['sector'].str.replace('sohna road road','sohna road')
df['sector'] = df['sector'].str.replace('malibu town','sector 47')
df['sector'] = df['sector'].str.replace('surat nagar 1','sector 104')
df['sector'] = df['sector'].str.replace('new colony','sector 7')
df['sector'] = df['sector'].str.replace('mianwali colony','sector 12')
df['sector'] = df['sector'].str.replace('jacobpura','sector 12')
df['sector'] = df['sector'].str.replace('rajiv nagar','sector 13')
df['sector'] = df['sector'].str.replace('ashok vihar','sector 3')

df['sector'] = df['sector'].str.replace('dlf phase 1','sector 26')
df['sector'] = df['sector'].str.replace('nirvana country','sector 50')
df['sector'] = df['sector'].str.replace('palam vihar','sector 2')
df['sector'] = df['sector'].str.replace('dlf phase 2','sector 25')
df['sector'] = df['sector'].str.replace('sushant lok phase 1','sector 43')
df['sector'] = df['sector'].str.replace('laxman vihar','sector 4')
df['sector'] = df['sector'].str.replace('dlf phase 4','sector 28')
df['sector'] = df['sector'].str.replace('dlf phase 3','sector 24')

df['sector'] = df['sector'].str.replace('sushant lok phase 3','sector 57')
df['sector'] = df['sector'].str.replace('dlf phase 5','sector 43')
df['sector'] = df['sector'].str.replace('rajendra park','sector 105')
df['sector'] = df['sector'].str.replace('uppals southend','sector 49')
df['sector'] = df['sector'].str.replace('sohna','sohna road')
df['sector'] = df['sector'].str.replace('ashok vihar phase 3 extension','sector 5')
df['sector'] = df['sector'].str.replace('south city 1','sector 41')
df['sector'] = df['sector'].str.replace('ashok vihar phase 2','sector 5')

df['sector'] = df['sector'].str.replace('sector 95a','sector 95')
df['sector'] = df['sector'].str.replace('sector 23a','sector 23')
df['sector'] = df['sector'].str.replace('sector 12a','sector 12')
df['sector'] = df['sector'].str.replace('sector 3a','sector 3')
df['sector'] = df['sector'].str.replace('sector 110 a','sector 110')
df['sector'] = df['sector'].str.replace('patel nagar','sector 15')
df['sector'] = df['sector'].str.replace('a block sector 43','sector 43')
df['sector'] = df['sector'].str.replace('maruti kunj','sector 12')
df['sector'] = df['sector'].str.replace('b block sector 43','sector 43')

df['sector'] = df['sector'].str.replace('sector-33 sohna road','sector 33')
df['sector'] = df['sector'].str.replace('sector 1 manesar','manesar')
df['sector'] = df['sector'].str.replace('sector 4 phase 2','sector 4')
df['sector'] = df['sector'].str.replace('sector 1a manesar','manesar')
df['sector'] = df['sector'].str.replace('c block sector 43','sector 43')
df['sector'] = df['sector'].str.replace('sector 89 a','sector 89')
df['sector'] = df['sector'].str.replace('sector 2 extension','sector 2')
df['sector'] = df['sector'].str.replace('sector 36 sohna road','sector 36')

df['sector'] = df['sector'].str.replace('sohna road road','sohna road')
df['sector'] = df['sector'].str.replace('sohna road road road','sohna road')
df['sector'] = df['sector'].str.replace('sohna road road  road','sohna road')
df['sector'] = df['sector'].str.replace('lajpat nagar','sector 12')
df['sector'] = df['sector'].str.replace('mayfield garden','sector 51')
df['sector'] = df['sector'].str.replace('surat nagar phase 2','sector 102')
df['sector'] = df['sector'].str.replace('shanti nagar','sector 11')

In [73]:
df['sector'] = df['sector'].str.strip()

In [78]:
df['sector'] = df['sector'].replace('sohna road road', 'sohna road')

In [94]:
df['society'] = df['society'].str.strip()

In [97]:
df[df['sector'] == 'new']

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
727,flat,4 BHK Flat in New Gurgaon,new,sare homes,0.85,4786.0,1776.0,Super Built up area 1776(165 sq.m.),4,4,3,not available,"New Gurgaon, Gurgaon, Haryana",3.0,NaN,5 to 10 Year Old,"['Columbia Asia Hospital', 'Apex Multi Special...",Located in the popular residential address of ...,[],NaN,NaN


In [86]:
df.loc[df['society']=='green court','sector'] = 'sector 90'

In [90]:
df.loc[df['society']=='dlf 76','sector'] = 'sector 76'

In [96]:
df.loc[df['society']=='takshila heights sector 37 c','sector'] = 'sector 37c'

In [91]:
df[df['society'] == 'dlf 76']

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
623,flat,4 BHK Flat in New Gurgaon,sector 76,dlf 76,4.0,11428.0,3500.0,Carpet area: 3500 (325.16 sq.m.),4,4,2,"study room,servant room","New Gurgaon, Gurgaon, Haryana",4.0,NaN,Jun 2027,"['Shri Balaji Hospital and Trauma Center', 'S....",This lovely 4 bhk apartment/flat in new gurgao...,"['6 Wardrobe', '1 Fridge', '8 Fan', '1 Exhaust...","['Security / Fire Alarm', 'Feng Shui / Vaastu ...",NaN


In [80]:
temp = pd.DataFrame(df['sector'].value_counts())
temp = temp.reset_index().rename(columns={'index':'sector_name'})
temp= temp.drop(columns='sector')
temp['sector_name'][~temp['sector_name'].str.contains('sector')]

0                                             sohna road
42                                               manesar
67                                           gwal pahari
93                                     dwarka expressway
108                                                  new
115                                 golf course ext road
120                                          mohan nagar
121                                             rosewood
124                                      bhawani enclave
125                                          sushant lok
126                                       sheetla colony
127                                          , delhi ncr
128                                     golf course road
129                                            new basti
130                                          iffco chowk
131                                        shankar vihar
132                                                 nars
133                            

In [105]:
df[df['sector'] == 'golf course ext road']

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating


In [102]:
df.iloc[950, 2] = 'sector 68'

In [104]:
df.iloc[1142, 2] = 'sector 67'

In [112]:
df[df['sector'] == 'rosewood']

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating


In [111]:
df.loc[df['sector'] == 'rosewood', 'sector'] = 'sector 49'

In [117]:
df[df['sector'] == 'bhawani enclave']

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating


In [116]:
df.loc[df['sector'] == 'bhawani enclave', 'sector'] = 'sector 9'

In [124]:
df[df['sector'] == 'sushant lok']

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating


In [120]:
df.loc[df['society'] == 'the ivy', 'sector'] = 'sector 28'

In [123]:
df.loc[df['society'] == 'surbhi apartment', 'sector'] = 'sector 43'

In [126]:
df[df['sector'] == 'sheetla colony']

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
1876,house,2 Bedroom House for sale in Sheetla Colony,sheetla colony,independent,0.70,7701.0,909.0,Plot area 101(84.45 sq.m.),2,2,0,NaN,House No 32 Block A Gali No 5 Sheetla Enclave ...,1.0,North-East,5 to 10 Year Old,"['Chintapurni Mandir', 'Sheetla Mata Mandir', ...","2 bath, semi-Furnished, ground floor (Of 1), o...","['1 Water Purifier', '5 Fan', '1 Exhaust Fan',...","['Private Garden / Terrace', 'Water purifier',...","['Environment3 out of 5', 'Lifestyle3 out of 5..."
1927,house,9 Bedroom House for sale in Sheetla Colony,sheetla colony,independent,1.65,75000.0,220.0,Built Up area: 220 (20.44 sq.m.),9,9,0,NaN,"Sheetla Colony, Sheetla Colony, Gurgaon, Haryana",1.0,NaN,0 to 1 Year Old,NaN,Best in class property available at sheetla co...,"['1 Wardrobe', '1 Light', 'No AC', 'No Bed', '...",NaN,"['Environment3 out of 5', 'Lifestyle3 out of 5..."


'House No 32 Block A Gali No 5 Sheetla Enclave Near Global Hospital Gurgaon, Sheetla Colony, Gurgaon, Haryana'

In [130]:
df[df['sector'] == ', delhi ncr']

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating


In [129]:
df.iloc[2554,2] = 'NA'
df.iloc[2883,2] = 'NA'

In [133]:
df[df['sector'] == 'golf course road']

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
2478,house,3 Bedroom House for sale in Golf Course Road,golf course road,independent,1.6,160000.0,100.0,Plot area 100(9.29 sq.m.),3,2,2,NaN,"Golf Course Road, Gurgaon, Haryana",41.0,NaN,1 to 5 Year Old,"['Sector 42-43 metro station', 'Sector 53-54 m...",We have a house/villa available for sale in go...,[],NaN,NaN


In [132]:
df.iloc[3218,2] = 'sector 62'

In [137]:
df[df['sector'] == 'new basti']

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating


In [136]:
df.loc[df['sector'] == 'new basti', 'sector'] = 'sector 8'

In [140]:
df[df['sector'] == 'iffco chowk']

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating


In [139]:
df.loc[df['sector'] == 'iffco chowk', 'sector'] = 'sector 28'

In [146]:
df[df['sector'] == 'shankar vihar']

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating


In [145]:
df.loc[df['sector'] == 'shankar vihar', 'sector'] = 'sector 104'

In [151]:
df[df['sector'] == 'nars']

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating


In [150]:
df.loc[df['sector'] == 'nars', 'sector'] = 'sector 37'

In [156]:
df[df['sector'] == 'rajiv chowk']

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating


In [155]:
df.loc[df['society'] == 'hans enclave', 'sector'] = 'sector 33'

In [159]:
df[df['sector'] == 'samaspur']

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating


In [158]:
df.loc[df['sector'] == 'samaspur', 'sector'] = 'sector 51'

In [162]:
df[df['sector'] == 'faridabad road']

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
3960,house,1 Bedroom House for sale in Gurgaon Faridabad ...,faridabad road,independent,0.35,3500.0,1000.0,Plot area 1000(92.9 sq.m.),1,4,1,NaN,"Gurgaon Faridabad Road, Gurgaon, Haryana",1.0,NaN,5 to 10 Year Old,NaN,Location and properties very good,NaN,NaN,"['Environment5 out of 5', 'Lifestyle4 out of 5..."


In [161]:
df.iloc[403,2] = 'gwal pahari'

In [165]:
df[df['sector'] == 'manohar nagar']

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating


In [164]:
df.loc[df['sector'] == 'manohar nagar', 'sector'] = 'sector 12'

In [166]:
df[df['sector'] == 'bissar']

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
207,house,3 Bedroom House for sale in Bissar,bissar,silverglades tarudhan valley golf resort,3.60,14815.0,2430.0,Plot area 270(225.75 sq.m.),3,3,2,"Pooja Room,Study Room,Servant Room,Others","Bissar, Gurgaon, Haryana",2.0,NaN,5 to 10 Year Old,NaN,This 3 bhk house for sale in bissar is an idea...,"['1 Water Purifier', '10 Fan', '1 Fridge', '1 ...","['Centrally Air Conditioned', 'Water purifier'...",NaN
491,house,3 Bedroom House for sale in Bissar,bissar,silverglades tarudhan valley golf resort,3.05,12551.0,2430.0,Plot area 270(225.75 sq.m.),3,3,2,"Pooja Room,Study Room,Servant Room,Others","Bissar, Gurgaon, Haryana",2.0,NaN,5 to 10 Year Old,NaN,Wishing to settle down in an individual house ...,"['1 Water Purifier', '9 Fan', '1 Fridge', '1 E...","['Centrally Air Conditioned', 'Water purifier'...",NaN


In [167]:
df[df['sector'] == 'surat nagar']

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
446,house,4 Bedroom House for sale in Surat Nagar,surat nagar,independent,0.71,10142.0,700.0,Plot area 700(65.03 sq.m.),4,2,1,NaN,"Surat Nagar, Gurgaon, Haryana",2.0,NaN,0 to 1 Year Old,"['Chintapurni Mandir', 'State bank ATM', 'Dr. ...",Interested to sell independent house/villa.It ...,NaN,NaN,"['Environment3 out of 5', 'Lifestyle3 out of 5..."
737,house,7 Bedroom House for sale in Surat Nagar,surat nagar,independent,0.95,7280.0,1305.0,Plot area 145(121.24 sq.m.),7,6,3+,NaN,"Surat Nagar, Gurgaon, Haryana",3.0,NaN,10+ Year Old,"['Chintapurni Mandir', 'State bank ATM', 'Dr. ...",Interested to sell independent house/villa.It ...,"['2 Wardrobe', '2 Fan', '2 Light', 'No AC', 'N...",NaN,"['Environment3 out of 5', 'Lifestyle3 out of 5..."


In [170]:
df[df['sector'] == 'sispal vihar']

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating


In [169]:
df.loc[df['sector'] == 'sispal vihar', 'sector'] = 'sector 49'

In [174]:
df[df['sector'] == 'daulatabad']

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
1948,house,2 Bedroom House for sale in Daulatabad,daulatabad,independent,0.7,7000.0,1000.0,Built Up area: 1000 (92.9 sq.m.)Carpet area: 5...,2,2,1,NaN,"Daulatabad, Gurgaon, Haryana",5.0,North-East,undefined,NaN,"2 bath, furnished, 2nd floor (Of 5), overlooki...","['1 Wardrobe', '1 Fan', '1 Light', 'No AC', 'N...",NaN,NaN


In [173]:
df.iloc[30,2] = 'sector 16a'

In [175]:
df.iloc[1948,2] = 'sector 106'

In [178]:
df[df['sector'] == 'ambience island']

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating


In [177]:
df.loc[df['sector'] == 'ambience island', 'sector'] = 'sector 24'

In [180]:
df[df['sector'] == 'farukhnagar']

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
710,house,3 Bedroom House for sale in Farukhnagar,farukhnagar,independent,0.52,5778.0,900.0,Plot area 100(83.61 sq.m.),3,2,2,NaN,"Farukhnagar, Gurgaon, Haryana",2.0,NaN,0 to 1 Year Old,"['Dr. Manish Pandey', 'Verma Dental Clinic', '...",Interested to sell independent house/villa.Pla...,[],NaN,"['Environment4 out of 5', 'Lifestyle4 out of 5..."
2499,house,6 Bedroom House for sale in Farukhnagar,farukhnagar,independent,2.30,25386.0,906.0,Plot area 906(84.17 sq.m.),6,6,0,Others,"Ward No 9, Farukhnagar, Gurgaon, Haryana",0.0,NaN,10+ Year Old,"['Dr. Manish Pandey', 'Shri Bala G Hospital an...",Property with an open area with 6 room and 6 b...,NaN,"['Private Garden / Terrace', 'Water Storage', ...","['Environment4 out of 5', 'Lifestyle4 out of 5..."


In [183]:
df[df['sector'] == 'acharya puri']

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating


In [182]:
df.loc[df['sector'] == 'acharya puri', 'sector'] = 'sector 12'

In [240]:
df.head()

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
0,house,2 Bedroom House for sale in Laxman Vihar,sector 4,independent,1.25,78125.0,160.0,Built Up area: 160 (14.86 sq.m.),2,3,0,NaN,"Laxman Vihar, Sector 3a, Laxman Vihar, Gurgaon...",1.0,NaN,undefined,"['Chintapurni Mandir', 'Sheetla Mata Mandir', ...",Best in class property available at laxman vih...,"['1 Light', 'No AC', 'No Bed', 'No Chimney', '...",NaN,"['Environment4 out of 5', 'Lifestyle4 out of 5..."
1,house,3 Bedroom House for sale in Sector 52 Gurgaon,sector 52,ardee city,2.25,16071.0,1400.0,Plot area 2160(200.67 sq.m.)Built Up area: 180...,3,3,3,Servant Room,"Upper First Floor, Sector 52 Gurgaon, Gurgaon,...",4.0,South-East,0 to 1 Year Old,"['Hanuman Mandir', 'New Life Church', 'Radhakr...",Brand new 3bhk independent upper first floor o...,"['3 Wardrobe', '7 Fan', '1 Exhaust Fan', '3 Ge...","['Private Garden / Terrace', 'Maintenance Staf...","['Environment4 out of 5', 'Safety4 out of 5', ..."
2,flat,3 BHK Flat in Sector 37D Gurgaon,sector 37d,signature global city,1.25,6082.0,2055.0,Carpet area: 2055 (190.92 sq.m.),3,3,3,not available,"Sector 37D Gurgaon, Gurgaon, Haryana",4.0,NaN,Dec 2024,"['Sapphire 83 Mall Sector 83', 'Dwarka Express...",It's a very beautiful property with amazing lo...,"['1 Modular Kitchen', '1 Chimney', '4 AC', 'No...","['Power Back-up', 'Lift(s)', 'Swimming Pool', ...","['Environment4 out of 5', 'Lifestyle4 out of 5..."
3,house,4 Bedroom House for sale in Palam Vihar,sector 2,ansal api spanish court,2.90,18626.0,1557.0,Plot area 173(144.65 sq.m.)Built Up area: 1415...,4,4,3,Store Room,"A, Block Ansals Palam Vihar, Palam Vihar, Gurg...",2.0,North,1 to 5 Year Old,"['Palam Vihar Vyapar kendra', 'Palam triangle'...",On sale 173 sq yrds built up house park facing...,"['4 Wardrobe', '10 Fan', '1 Exhaust Fan', '2 G...","['Feng Shui / Vaastu Compliant', 'Maintenance ...","['Environment5 out of 5', 'Lifestyle5 out of 5..."
4,flat,2 BHK Flat in Sector 89 Gurgaon,sector 89,smart world gems,0.86,7796.0,1103.0,Carpet area: 1103 (102.47 sq.m.),2,3,2,"study room,pooja room,servant room","Sector 89 Gurgaon, Gurgaon, Haryana",4.0,North-East,Within 3 months,"['Sant Soordas Sihi Metro Station', 'Airia Mal...","Park facing, green belt, at very reasonable ra...","['2 Wardrobe', '3 Fan', '1 Exhaust Fan', '3 Ge...","['Security / Fire Alarm', 'Feng Shui / Vaastu ...","['Environment4 out of 5', 'Safety4 out of 5', ..."


In [338]:
df.sample(5)

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
1982,flat,2 BHK Flat in Sohna,sohna road,trisara our homes 3,0.34,5405.0,629.0,Carpet area: 629 (58.44 sq.m.),2,2,1,not available,"Sohna, Gurgaon, Haryana",0.0,NaN,0 to 1 Year Old,"['Metro Station', 'Gyanchand Market', 'Rajiv C...",It beautiful construction and design,NaN,NaN,"['Environment4 out of 5', 'Safety4 out of 5', ..."
671,flat,3 BHK Flat in Sector 69 Gurgaon,sector 69,tulip violet,1.38,8972.0,1538.0,Carpet area: 1538 (142.88 sq.m.),3,3,1,not available,"Sector 69 Gurgaon, Gurgaon, Haryana",12.0,West,1 to 5 Year Old,"['Airia Mall Sector 68', 'Southern Peripheral ...",Check out this 3 bhk apartment for sale in tul...,"['1 Modular Kitchen', 'No AC', 'No Bed', 'No C...","['Centrally Air Conditioned', 'Water purifier'...","['Green Area5 out of 5', 'Construction4 out of..."
3722,flat,6 BHK Flat in Sector 70 Gurgaon,sector 70,tulip ivory,5.00,10416.0,4800.0,Carpet area: 4800 (445.93 sq.m.),6,7,3+,"study room,servant room","Sector 70 Gurgaon, Gurgaon, Haryana",0.0,NaN,1 to 5 Year Old,"['Reach 3 Roads Shopping Mall', 'Southern Peri...",It is new unused property park facing with kit...,[],"['Security / Fire Alarm', 'Feng Shui / Vaastu ...","['Green Area3.5 out of 5', 'Amenities4 out of ..."
1673,house,5 Bedroom House for sale in DLF Phase 1,dlf phase 1,independent,20.00,44444.0,4500.0,Plot area 500(418.06 sq.m.),5,7,3+,"Servant Room,Pooja Room","DLF Phase 1, Gurgaon, Haryana",3.0,West,1 to 5 Year Old,"['Guru dronacharya metro station', 'Dlf phase ...","West facing kothi , area- 500 sq yard, \n5bhk-...","['1 Water Purifier', '16 Fan', '1 Fridge', '1 ...","['Security / Fire Alarm', 'Private Garden / Te...","['Environment5 out of 5', 'Lifestyle5 out of 5..."
3169,flat,3 BHK Flat in Sohna,sohna road,eldeco accolade,1.14,6530.0,1746.0,Super Built up area 1751(162.67 sq.m.)Carpet a...,3,3,3+,not available,"P-405, Sohna, Gurgaon, Haryana",4.0,East,1 to 5 Year Old,"['Global City Centre', 'Sohna Road', 'Damdama ...",The eldeco group has been at the forefront of ...,"['3 Wardrobe', '1 Modular Kitchen', '1 Chimney...","['Centrally Air Conditioned', 'Water purifier'...","['Green Area5 out of 5', 'Construction4.5 out ..."


In [246]:
df.iloc[2398,2]

'sector 33'

In [245]:
df.iloc[2398,2] = 'sector 33'

In [247]:
df.iloc[2498,2] = 'sector 54'

In [248]:
df.iloc[2982,2] = 'sector 37c'

In [249]:
df.iloc[1085,2] = 'sector 1'

In [250]:
df.iloc[1034,2] = 'sector 11'

In [251]:
df.iloc[2821,2] = 'sector 8'

In [252]:
df.iloc[291,2] = 'sector 61'

In [253]:
df.iloc[3018,2] = 'sector 28'

In [254]:
df['sector'] = data['sector'].replace('uppals southend', 'sector 49')

In [255]:
df.iloc[830,2] = 'sector 49'

In [256]:
df.iloc[830,2]

'sector 49'

In [257]:
df.iloc[2489,2] = 'sector 104'

In [258]:
df.iloc[1326,2] = 'sector 110'

In [259]:
df.iloc[3895,2] = 'sector 13'

In [260]:
df.iloc[482,2] = 'sector 3'

In [261]:
df.iloc[1860,2] = 'sector 3'

In [262]:
df.iloc[540,2] = 'sector 61'

In [263]:
df.iloc[1891,2] = 'sector 50'

In [264]:
df.iloc[2746,2] = 'sector 3a'

In [265]:
df.iloc[2470,2] = 'sector 9'

In [266]:
df.iloc[809,2] = 'sohna road'

In [267]:
df.iloc[3396,2] = 'sector 1'

In [268]:
df.iloc[1926,2] = 'sector 3'

In [270]:
df.iloc[3643,2] = 'sector 7'

In [273]:
df['sector'] = df['sector'].str.replace('nirvana country', 'sector 50')

In [274]:
df.iloc[1295,2] = 'sector 33'

In [277]:
df.iloc[1044,2]

' sector 50'

In [281]:
df['sector'] = df['sector'].str.replace('sector-33 sohna road', 'sector 33')

In [282]:
df['sector'] = df['sector'].str.replace('uppals southend', 'sector 49')

In [283]:
df.iloc[1306,2]

' sector 49'

In [285]:
df['sector'] = df['sector'].str.replace('sadar bazar', 'sector 7')

In [296]:
df.iloc[2101,2]

' sector 8'

In [290]:
df['sector'] = df['sector'].str.replace('madanpuri', 'sector 7')

In [293]:
df.iloc[1865,2] = 'sector 24'

In [295]:
df['sector'] = df['sector'].str.replace('new basti', 'sector 8')

In [302]:
df.iloc[2888,2] = 'sector 30'

In [303]:
df.iloc[3141,2] = 'sector 5'

In [306]:
df['sector'] = df['sector'].str.replace('manohar nagar', 'sector 12')

In [310]:
df.loc[df['society'] == 'nirvana country', 'sector'] = 'sector 50'

In [314]:
df.iloc[48,2] = 'sector 43'

In [316]:
df.iloc[16,2] = 'sector 11'

In [318]:
df['sector'] = df['sector'].str.replace('maruti kunj', 'sector 67')

In [326]:
df.iloc[3158,2] = 'sector 28'

In [329]:
df.iloc[16,2] = 'sector 45'

In [331]:
df['sector'] = df['sector'].str.replace('sohna road road', 'sohna road')

In [333]:
df.iloc[3608,2] = 'sector 4'

In [336]:
df.iloc[1302,2] = 'sector 36'

In [339]:
df.iloc[1982,2] = 'sector 6'

In [342]:
df[~df['sector'].str.contains('sector')]['sector'].value_counts()

 sohna road                                 169
 palam vihar                                 50
 dlf phase 1                                 44
 dlf phase 2                                 42
 sushant lok phase 1                         31
 laxman vihar                                26
 dlf phase 4                                 25
 dlf phase 3                                 22
 sushant lok phase 3                         16
 dlf phase 5                                 14
 rajendra park                               14
 gwal pahari                                 14
 laxman vihar phase 2                        13
 ashok vihar phase 3 extension               11
 south city 1                                 9
 c block sushant lok phase 1                  9
 malibu town                                  8
 ashok vihar phase 2                          8
 dwarka expressway                            8
 palam vihar extension                        8
 mehrauli  road                         

In [343]:
df.head(3)

,property_type,property_name,sector,society,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating
0,house,2 Bedroom House for sale in Laxman Vihar,laxman vihar,independent,1.25,78125.0,160.0,Built Up area: 160 (14.86 sq.m.),2,3,0,NaN,"Laxman Vihar, Sector 3a, Laxman Vihar, Gurgaon...",1.0,NaN,undefined,"['Chintapurni Mandir', 'Sheetla Mata Mandir', ...",Best in class property available at laxman vih...,"['1 Light', 'No AC', 'No Bed', 'No Chimney', '...",NaN,"['Environment4 out of 5', 'Lifestyle4 out of 5..."
1,house,3 Bedroom House for sale in Sector 52 Gurgaon,sector 52,ardee city,2.25,16071.0,1400.0,Plot area 2160(200.67 sq.m.)Built Up area: 180...,3,3,3,Servant Room,"Upper First Floor, Sector 52 Gurgaon, Gurgaon,...",4.0,South-East,0 to 1 Year Old,"['Hanuman Mandir', 'New Life Church', 'Radhakr...",Brand new 3bhk independent upper first floor o...,"['3 Wardrobe', '7 Fan', '1 Exhaust Fan', '3 Ge...","['Private Garden / Terrace', 'Maintenance Staf...","['Environment4 out of 5', 'Safety4 out of 5', ..."
2,flat,3 BHK Flat in Sector 37D Gurgaon,sector 37d,signature global city,1.25,6082.0,2055.0,Carpet area: 2055 (190.92 sq.m.),3,3,3,not available,"Sector 37D Gurgaon, Gurgaon, Haryana",4.0,NaN,Dec 2024,"['Sapphire 83 Mall Sector 83', 'Dwarka Express...",It's a very beautiful property with amazing lo...,"['1 Modular Kitchen', '1 Chimney', '4 AC', 'No...","['Power Back-up', 'Lift(s)', 'Swimming Pool', ...","['Environment4 out of 5', 'Lifestyle4 out of 5..."


In [346]:
data['sector'] = data['sector'].str.strip()

In [344]:
df.shape

(3974, 21)

In [347]:
df.to_csv('gurgaon_properties_cleaned_v1.csv', index=False)